In [48]:
import numpy as np
import pandas as pd
df1=pd.read_csv("data_se.csv")
df1.head()

,start_city_id,is_cross_city_flag,is_station_flag,consult_hour,byway_degree,final_rank,travel_to_call,answer_to_call,travel_to_answer,is_td_cancel_aft_flag,...,gender_p_F,gender_d_F,is_auto_deal_闪电确认,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,84,1,0,12,90,0.0,0.801667,0.413611,0.388056,1,...,0,0,0,0,0,0,0,1,0,0
1,6,1,0,8,95,0.0,9.384722,8.913333,0.471389,1,...,1,0,1,0,0,0,0,0,0,1
2,6,0,0,11,95,0.0,0.401111,0.337778,0.063333,0,...,1,0,1,0,1,0,0,0,0,0
3,5,0,0,7,90,0.0,10.689167,0.030833,10.658333,0,...,1,0,1,1,0,0,0,0,0,0
4,92,0,0,16,90,0.0,0.394722,0.093889,0.300833,0,...,1,0,0,0,0,0,1,0,0,0


In [49]:
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np
import pandas as pd
import os
import seaborn as sea
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit,ShuffleSplit,StratifiedKFold,KFold
import numpy as np
import pandas as pd
import os
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from xgboost import plot_importance
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit,ShuffleSplit,StratifiedKFold,KFold
from xgboost.sklearn import XGBClassifier
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import joblib
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
#from sklearn.externals import joblib
from sklearn.model_selection import cross_val_score



In [73]:
df1=pd.read_csv("data_se.csv")
df1.head()
#df1.isnull().sum()  #只有车主顺路程度有缺失，其结果为01离散分布，对确实值填充
df1=df1.fillna(0)
Y = df1['is_td_cancel_aft_flag']
X =df1.drop(['is_td_cancel_aft_flag'],axis=1,inplace=False)
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=0)
    #X.shape,y.shape,X_train.shape,X_test.shape,y_train.shape,y_test.shape
#区分闪电确认

df_y=df1[df1["is_auto_deal_闪电确认"]==1]
df_n=df1[df1["is_auto_deal_闪电确认"]==0]
Y1 = df_y['is_td_cancel_aft_flag']
X1 =df_y.drop(['is_td_cancel_aft_flag',"is_auto_deal_闪电确认"],axis=1,inplace=False)
X1_train,X1_test,y1_train,y1_test = train_test_split(X1,Y1,test_size=0.3,random_state=0)
    #X.shape,y.shape,X_train.shape,X_test.shape,y_train.shape,y_test.shape
Y2 = df_n['is_td_cancel_aft_flag']
X2 =df_n.drop(['is_td_cancel_aft_flag'],axis=1,inplace=False)
X2_train,X2_test,y2_train,y2_test = train_test_split(X2,Y2,test_size=0.3,random_state=0)
X2_train.head()

,start_city_id,is_cross_city_flag,is_station_flag,consult_hour,byway_degree,final_rank,travel_to_call,answer_to_call,travel_to_answer,is_carp_flag,...,gender_p_F,gender_d_F,is_auto_deal_闪电确认,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
51331,42,0,0,17,95,0.0,0.903333,0.608333,0.295000,1,...,1,0,0,0,0,0,0,1,0,0
1591,3,0,0,17,85,0.0,0.493333,0.018056,0.475278,1,...,0,0,0,0,0,1,0,0,0,0
46433,1,0,0,16,95,0.0,3.051944,0.373611,2.678333,1,...,1,0,0,0,0,1,0,0,0,0
76984,54,0,0,8,80,0.0,8.517778,0.512778,8.005000,1,...,1,0,0,0,0,0,0,1,0,0
39287,19,0,0,12,90,0.0,0.539444,0.014722,0.524722,1,...,0,0,0,0,1,0,0,0,0,0


In [74]:
#xgb
def xgb(X_train, y_train,X_test, y_test):
    xgbm = XGBClassifier()
    xgbm.fit(X_train, y_train)

    # predicted proability
    train_pred = xgbm.predict_proba(X_train)[:,1]
    test_pred = xgbm.predict_proba(X_test)[:,1]
    # 预测结果
    y_train_pred = xgbm.predict(X_train) 
    y_test_pred = xgbm.predict(X_test)

    # MSE 均方误差
    print('准确率:',xgbm.score(X_test, y_test)) # 全体
    print('ACC: %.4f' % metrics.accuracy_score(y_test,y_test_pred)) 
    print('MSE:', metrics.mean_squared_error(y_test,y_test_pred))
    print('AUC: %.4f' % metrics.roc_auc_score(y_test,test_pred))
    print('Recall: %.4f' % metrics.recall_score(y_test,y_test_pred))  
    print('Precesion: %.4f' % metrics.precision_score(y_test,y_test_pred))  # y
    print('F1-score: %.4f' % metrics.f1_score(y_test,y_test_pred)) 

    fpr,tpr,threshold=metrics.roc_curve(y_test,test_pred)
    rocauc=metrics.auc(fpr,tpr)   #AUC，AUC>0.7，表示可接受的区辨能力
    print("AUC:",rocauc)
    print("KS:",max(tpr-fpr))  # KS值，KS>0.3，表示可接受的区辨能力
    importance = pd.DataFrame({'features':X_train.columns,'importance':xgbm.feature_importances_}) 
    return importance.sort_values(['importance'],ascending=False).reset_index(drop=True)


In [75]:
#闪电确认 
ax_s=xgb(X1_train, y1_train,X1_test, y1_test)
ax_s=ax_s.reset_index(drop=True)
ax_s

准确率: 0.7550891655450875
ACC: 0.7551
MSE: 0.24491083445491252
AUC: 0.7051
Recall: 0.1413
Precesion: 0.6272
F1-score: 0.2307
AUC: 0.705058889177948
KS: 0.30460346424049817


,features,importance
0,dri_cancel_rate,0.126198
1,navi_dis_km_route,0.106509
2,pas_finish_ord_cnt,0.097622
3,pas_cancel_rate,0.077800
4,consult_hour_type_早高峰(07:00-08:59),0.073379
5,answer_to_call,0.051551
6,travel_to_call,0.043483
7,is_td_answer_carp_flag,0.032641
8,is_cross_city_flag,0.031151
9,consult_hour,0.030763


In [76]:
#lgb
def lgb(X_train, y_train,X_test, y_test):
      #X.shape,y.shape,X_train.shape,X_test.shape,y_train.shape,y_test.shape
    lgbm = LGBMClassifier(max_depth=6,n_estimators=1000)
    lgbm.fit(X_train.values, y_train.values)
    # predicted proability
    train_pred = lgbm.predict_proba(X_train)[:,1]
    test_pred = lgbm.predict_proba(X_test)[:,1]
    # 预测结果
    y_train_pred = lgbm.predict(X_train) 
    y_test_pred = lgbm.predict(X_test)

    # MSE 均方误差
     # MSE 均方误差
    print('准确率:',lgbm.score(X_test, y_test)) # 全体
    print('ACC: %.4f' % metrics.accuracy_score(y_test,y_test_pred)) 

    fpr,tpr,threshold=metrics.roc_curve(y_test,test_pred)
    rocauc=metrics.auc(fpr,tpr)   #AUC，AUC>0.7，表示可接受的区辨能力
    print("AUC:",rocauc)
    print("KS:",max(tpr-fpr))  # KS值，KS>0.3，表示可接受的区辨能力
  
    print(confusion_matrix(y_test,y_test_pred))

    # 每个系数的影响力，系数越大表示特征在分类中起到的作用越大
    importance = pd.DataFrame({'features':X_train.columns,'importance':lgbm.feature_importances_}) 
    return importance.sort_values(['importance'],ascending=False)


In [77]:
#闪电确认 
lg_s=lgb(X1_train, y1_train,X1_test, y1_test)
lg_s=lg_s.reset_index(drop=True)
lg_s

准确率: 0.74810733512786
ACC: 0.7481
AUC: 0.6885112563287411
KS: 0.2758584814068285
[[16444  1155]
 [ 4834  1343]]


,features,importance
0,navi_dis_km_route,2004
1,travel_to_publish,1974
2,answer_to_call,1932
3,travel_to_answer,1689
4,dri_cancel_rate,1557
5,start_city_id,1553
6,travel_to_call,1371
7,age_d,1365
8,d_blame_canel_rate,1346
9,age_p,1330


In [78]:
#决策树
from sklearn.tree import DecisionTreeClassifier,plot_tree#决策树
def dct(X_train, y_train,X_test, y_test):
    #X.shape,y.shape,X_train.shape,X_test.shape,y_train.shape,y_test.shape
    dct = DecisionTreeClassifier(max_depth=6)
    dct.fit(X_train.values, y_train.values)
    # predicted proability
    train_pred = dct.predict_proba(X_train)[:,1]
    test_pred = dct.predict_proba(X_test)[:,1]
    # 预测结果
    y_train_pred = dct.predict(X_train) 
    y_test_pred = dct.predict(X_test)

    # MSE 均方误差
    print('准确率:',dct.score(X_test, y_test)) # 全体
    print('ACC: %.4f' % metrics.accuracy_score(y_test,y_test_pred)) 

    fpr,tpr,threshold=metrics.roc_curve(y_test,test_pred)
    rocauc=metrics.auc(fpr,tpr)   #AUC，AUC>0.7，表示可接受的区辨能力
    print("AUC:",rocauc)
    print("KS:",max(tpr-fpr))  # KS值，KS>0.3，表示可接受的区辨能力
  
    print(confusion_matrix(y_test,y_test_pred))

    # 每个系数的影响力，系数越大表示特征在分类中起到的作用越大
    importance = pd.DataFrame({'features':X_train.columns,'importance':dct.feature_importances_}) 
    return importance.sort_values(['importance'],ascending=False)


In [79]:
#闪电确认 
tree_s=dct(X1_train, y1_train,X1_test, y1_test)
tree_s=tree_s.reset_index(drop=True)
tree_s

准确率: 0.7457940780619112
ACC: 0.7458
AUC: 0.669321326712687
KS: 0.24440824935019423
[[17156   443]
 [ 5601   576]]


,features,importance
0,dri_cancel_rate,0.332585
1,answer_to_call,0.161341
2,pas_finish_ord_cnt,0.150621
3,pas_cancel_rate,0.114800
4,navi_dis_km_route,0.097624
5,travel_to_answer,0.019324
6,pas_cancel_aft_p_blame_cnt,0.018103
7,consult_hour,0.018065
8,travel_to_publish,0.013870
9,travel_to_call,0.012646


In [80]:
# 逻辑回归
from sklearn.linear_model import LogisticRegression 
def lr(X_train,Y_train,X_test, Y_test):
    lr= LogisticRegression(random_state=123)
    lr.fit(X_train,Y_train)
    # Y_pre = log.predict(X_test)#测试集预测
    acc_log_test = round(lr.score(X_test, Y_test) * 100, 2)#测试集上的评分
    print(acc_log_test)
    # 变量重要性排序
    coef_lr = pd.DataFrame({'var' : X_train.columns,
                            'coef' : lr.coef_.flatten()
                            })

    index_sort =  np.abs(coef_lr['coef']).sort_values(ascending=False).index
    coef_lr_sort = coef_lr.loc[index_sort,:].reset_index(drop=True)
    return coef_lr_sort

In [81]:
#闪电确认 ACC:
lr_s=lr(X1_train,y1_train,X1_test, y1_test)
lr_s["abs_coef"]=lr_s["coef"].apply(lambda x:abs(x))
lr_s=lr_s.sort_values(['abs_coef'],ascending=False).reset_index(drop=True)
lr_s

/home/xiaoju/jupyterlab/venv/lib64/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


74.74


,var,coef,abs_coef
0,consult_hour,0.026121,0.026121
1,pas_cancel_aft_p_blame_cnt,0.020397,0.020397
2,answer_to_call,0.018318,0.018318
3,pas_cancel_aft_p_ord_cnt,0.017006,0.017006
4,byway_degree,-0.014822,0.014822
5,pas_cancel_aft_ord_cnt,0.014228,0.014228
6,travel_to_answer,-0.011797,0.011797
7,pas_finish_ord_cnt,-0.011570,0.011570
8,age_p,-0.009145,0.009145
9,is_td_answer_carp_flag,-0.008790,0.008790


In [82]:
# ADABOOST
from sklearn.ensemble import AdaBoostClassifier
def ada(X_train,Y_train,X_test, Y_test):
    clf = AdaBoostClassifier(n_estimators=1000,random_state=123)
    clf.fit(X_train, Y_train)
    # Y_pre = log.predict(X_test)#测试集预测
    acc_log_test = round(clf.score(X_test, Y_test) * 100, 2)#测试集上的评分
    print(acc_log_test)
    # 变量重要性排序
    coef_lr = pd.DataFrame({'var' : X_train.columns,
                            'coef' : clf.feature_importances_
                            })

    index_sort =  np.abs(coef_lr['coef']).sort_values(ascending=False).index
    coef_lr_sort = coef_lr.loc[index_sort,:].reset_index(drop=True)
    return coef_lr_sort

In [ ]:
#闪电确认 ACC:
a1=ada(X1_train,y1_train,X1_test, y1_test)
a1["abs_coef"]=a1["coef"].apply(lambda x:abs(x))
a1=a1.sort_values(['abs_coef'],ascending=False).reset_index(drop=True)
ada_s=a1

In [ ]:
lg_s["rate_importance"]=round(lg_s["importance"]/lg_s["importance"].sum(),4)
pd.concat([ax_s,lg_s,tree_s,lr_s,ada_s],axis=1).to_csv("闪电确认特征.csv",encoding='utf_8_sig')